# RBF Calculation Speed-Up

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from krr import KRR, rbf_derivative
from sklearn.kernel_ridge import KernelRidge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import rbf_kernel

import cProfile
from rbf_derivative_cy import rbf_derivative as rbf_derivative_cy

### Get Data

In [2]:
random_state = 123
num_points = 1000
x_data = np.arange(0, num_points)
y_data = np.sin(x_data)

# split into training and testing
train_prnt = 0.7
x_train, x_test, y_train, y_test = \
    train_test_split(x_data, y_data,
                     train_size=train_prnt,
                     random_state=random_state)

/Users/eman/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
# make a new axis D [N x D]
x_train, x_test = x_train[:, np.newaxis], x_test[:, np.newaxis]
y_train, y_test = y_train[:, np.newaxis], y_test[:, np.newaxis]

# remove the mean from y training
y_train = y_train - np.mean(y_train)

param_grid = {"alpha": [1e0, 1e-1, 1e-2, 1e-3],
              "gamma": np.logspace(-2, 2, 5)}

# initialize the kernel ridge regression model
KRR_model = GridSearchCV(KernelRidge(kernel='rbf'), 
                         cv=5, param_grid=param_grid)

# fit model to data
KRR_model.fit(x_train, y_train)

# predict using the krr model
y_pred = KRR_model.predict(x_test)

# RBF Derivative

In [4]:
def rbf_derivative(x_train, x_function, weights, kernel_mat,
                   n_derivative=1, gamma=1.0):
    """This function calculates the rbf derivative
    Parameters
    ----------
    x_train : array, [N x D]
        The training data used to find the kernel model.

    x_function  : array, [M x D]
        The test points (or vector) to use.

    weights   : array, [N x D]
        The weights found from the kernel model
            y = K * weights

    kernel_mat: array, [N x M], default: None
        The rbf kernel matrix with the similarities between the test
        points and the training points.

    n_derivative : int, (default = 1) {1, 2}
        chooses which nth derivative to calculate

    gamma : float, default: None
        the parameter for the rbf_kernel matrix function

    Returns
    -------

    derivative : array, [M x D]
        returns the derivative with respect to training points used in
        the kernel model and the test points.

    Information
    -----------
    Author: Juan Emmanuel Johnson
    Email : jej2744@rit.edu
            juan.johnson@uv.es
    """

    # initialize rbf kernel
    derivative = np.zeros(np.shape(x_function))

    # consolidate the parameters
    theta = 2 * gamma

    # 1st derivative
    if n_derivative == 1:

        # loop through dimensions
        for dim in np.arange(0, np.shape(x_function)[1]):

            # loop through the number of test points
            for iTest in np.arange(0, np.shape(x_function)[0]):

                # loop through the number of test points
                for iTrain in np.arange(0, np.shape(x_train)[0]):

                    # calculate the derivative for the test points
                    derivative[iTest, dim] += theta * weights[iTrain] * \
                                              (x_train[iTrain, dim] -
                                               x_function[iTest, dim]) * \
                                              kernel_mat[iTrain, iTest]

    # 2nd derivative
    elif n_derivative == 2:

        # loop through dimensions
        for dim in np.arange(0, np.shape(x_function)[1]):

            # loop through the number of test points
            for iTest in np.arange(0, np.shape(x_function)[0]):

                # loop through the number of test points
                for iTrain in np.arange(0, np.shape(x_train)[0]):
                    derivative[iTest, dim] += weights[iTrain] * \
                                              (theta ** 2 *
                                               (x_train[iTrain, dim] - x_function[iTest, dim]) ** 2
                                               - theta) * \
                                              kernel_mat[iTrain, iTest]

    return derivative


In [5]:
# extract necessary parameters
gamma = KRR_model.best_params_['gamma']
weights = KRR_model.best_estimator_.dual_coef_
kernel = rbf_kernel(x_train, gamma=gamma)
n_derivative = 1

In [6]:
derivative = rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, n_derivative=n_derivative, gamma=gamma)

In [7]:
derivative_cy = rbf_derivative_cy(np.float64(x_train), 
                                  np.float64(x_test), 
                                  weights=weights.squeeze(), 
                                  kernel_mat=kernel, 
                                  n_derivative=n_derivative, 
                                  gamma=gamma)

In [8]:
print(np.array_equal(derivative, derivative_cy))
print(derivative.dtype)
print(derivative_cy.dtype)

True
float64
float64


### Timing the Current Function

In [39]:
original_python = %timeit -o rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, n_derivative=n_derivative, gamma=gamma)

1.45 s ± 126 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
#cProfile.run('rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, n_derivative=n_derivative, gamma=gamma)')
%prun rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, n_derivative=n_derivative, gamma=gamma)

In [11]:
%load_ext line_profiler

In [12]:
%lprun -f rbf_derivative rbf_derivative(x_train, x_test, weights=weights, kernel_mat=kernel, gamma=gamma)

In [13]:
from numba import double
from numba import jit, autojit, njit

In [14]:
rbf_derivative_numba = jit()(rbf_derivative)

In [15]:
# compile it the first time
derivative_numba = rbf_derivative_numba(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)

In [22]:
print('Numba w/ Jit:')
jit_speedup = %timeit -o rbf_derivative_numba(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)

Numba w/ Jit:
1.36 s ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
print(original_python.best / jit_speedup.best)

1.013040010240821


In [18]:
%load_ext cython

### With GIL Lock

In [46]:
%%cython -a

cimport cython
import numpy as np
cimport numpy as np

@cython.boundscheck(False)
@cython.wraparound(False)
def rbf_derivative_cython(np.float64_t[:, :] x_train, 
                   np.float64_t[:, :] x_function,
                   np.float64_t[:] weights,
                   np.float64_t[:, :] kernel_mat,
                   np.int_t n_derivative,
                   np.float64_t gamma):
    """This function calculates the rbf derivative using
    Cython. It has been fairly optimized and provides x100
    speedup over the original python function.
    
    Parameters
    ----------
    x_train : array, [N x D], int64
        The training data used to find the kernel model.

    x_function  : array, [M x D], int64
        The test points (or vector) to use.

    weights   : array, [N x D], float64
        The weights found from the kernel model
            y = K * weights

    kernel_mat: array, [N x M], float64
        The rbf kernel matrix with the similarities between the test
        points and the training points.

    n_derivative : int, (default = 1) {1, 2}, int
        chooses which nth derivative to calculate

    gamma : float, default: None, float64
        the parameter for the rbf_kernel matrix function

    Returns
    -------

    derivative : array, [M x D]
        returns the derivative with respect to training points used in
        the kernel model and the test points.

    Information
    -----------
    Author: Juan Emmanuel Johnson
    Email : jej2744@rit.edu
            juan.johnson@uv.es
    """
    cdef int d_dimensions = x_function.shape[1]
    cdef int n_test = x_function.shape[0]
    cdef int n_train = x_train.shape[0]
    cdef int idim, iTest, iTrain
    
    # initialize the derivative
    cdef np.float64_t[:,:] derivative = np.zeros((n_test, d_dimensions))

    # consolidate the parameters
    cdef np.float64_t theta = 2.0 * gamma


    if n_derivative == 1:
        
        # loop through dimensions
        for idim in range(d_dimensions):

            # loop through the number of test points
            for iTest in range(n_test):

                # loop through the number of test points
                for iTrain in range(n_train):

                    # calculate the derivative for the test points
                    derivative[iTest, idim] += theta * weights[iTrain] * \
                                              (x_train[iTrain, idim] -
                                               x_function[iTest, idim]) * \
                                              kernel_mat[iTrain, iTest]
                        
    # 2nd derivative
    elif n_derivative == 2:

        # loop through dimensions
        for dim in range(d_dimensions):

            # loop through the number of test points
            for iTest in range(n_test):

                # loop through the number of test points
                for iTrain in range(n_train):
                    derivative[iTest, dim] += weights[iTrain] * \
                                              (theta ** 2 *
                                               (x_train[iTrain, dim] - x_function[iTest, dim]) ** 2
                                               - theta) * \
                                              kernel_mat[iTrain, iTest] 
    else:
        raise ValueError('n_derivative should be equal to 1 or 2.')
                            
    return np.asarray(derivative)

In [47]:

derivative_cython = rbf_derivative_cython(np.float64(x_train),
                                          np.float64(x_test),
                                          kernel_mat=kernel, 
                                          weights=weights.squeeze(), 
                                          n_derivative=n_derivative, 
                                          gamma=np.float64(gamma))

In [48]:
print(np.allclose(derivative, derivative_cython))
print(derivative.dtype)
print(derivative_cy.dtype)

True
float64
float64


In [49]:
print('Cython:')
cython_speedup = %timeit -o rbf_derivative_cython(np.float64(x_train), np.float64(x_test), kernel_mat=kernel, weights=weights.squeeze(), n_derivative=n_derivative, gamma=gamma)

Cython:
389 µs ± 25.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [50]:
print('Pure Python:')
print(original_python.best / original_python.best) 

print('Numba w/ Jit Speedup:')
print(original_python.best / jit_speedup.best)

print('Cython speedup:')
print(original_python.best / cython_speedup.best)

Pure Python:
1.0
Numba w/ Jit Speedup:
1.0067572125699529
Cython speedup:
3772.616076209928


### Without GIL Lock

In [35]:
%%cython

cimport cython
import numpy as np
cimport numpy as np
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
def rbf_derivative_cython(np.float64_t[:, :] x_train, 
                   np.float64_t[:, :] x_function,
                   np.float64_t[:] weights,
                   np.float64_t[:, :] kernel_mat,
                   np.int_t n_derivative,
                   np.float64_t gamma):
    """This function calculates the rbf derivative using
    Cython. It has been fairly optimized and provides x100
    speedup over the original python function.
    
    Parameters
    ----------
    x_train : array, [N x D], int64
        The training data used to find the kernel model.

    x_function  : array, [M x D], int64
        The test points (or vector) to use.

    weights   : array, [N x D], float64
        The weights found from the kernel model
            y = K * weights

    kernel_mat: array, [N x M], float64
        The rbf kernel matrix with the similarities between the test
        points and the training points.

    n_derivative : int, (default = 1) {1, 2}, int
        chooses which nth derivative to calculate

    gamma : float, default: None, float64
        the parameter for the rbf_kernel matrix function

    Returns
    -------

    derivative : array, [M x D]
        returns the derivative with respect to training points used in
        the kernel model and the test points.

    Information
    -----------
    Author: Juan Emmanuel Johnson
    Email : jej2744@rit.edu
            juan.johnson@uv.es
    """
    cdef int d_dimensions = x_function.shape[1]
    cdef int n_test = x_function.shape[0]
    cdef int n_train = x_train.shape[0]
    cdef int idim, iTest, iTrain
    
    # initialize the derivative
    cdef np.float64_t[:,:] derivative = np.zeros((n_test, d_dimensions))

    # consolidate the parameters
    cdef np.float64_t theta = 2.0 * gamma


    if n_derivative == 1:
        
        # loop through dimensions
        for idim in prange(d_dimensions, nogil=True):

            # loop through the number of test points
            for iTest in prange(n_test, nogil=True):

                # loop through the number of test points
                for iTrain in prange(n_train, nogil=True):

                    # calculate the derivative for the test points
                    derivative[iTest, idim] += theta * weights[iTrain] * \
                                              (x_train[iTrain, idim] -
                                               x_function[iTest, idim]) * \
                                              kernel_mat[iTrain, iTest]
                        
    # 2nd derivative
    elif n_derivative == 2:

        # loop through dimensions
        for dim in prange(d_dimensions, nogil=True):

            # loop through the number of test points
            for iTest in prange(n_test, nogil=True):

                # loop through the number of test points
                for iTrain in prange(n_train, nogil=True):
                    derivative[iTest, dim] += weights[iTrain] * \
                                              (theta ** 2 *
                                               (x_train[iTrain, dim] - x_function[iTest, dim]) ** 2
                                               - theta) * \
                                              kernel_mat[iTrain, iTest] 
    else:
        raise ValueError('n_derivative should be equal to 1 or 2.')
                            
    return np.asarray(derivative)


Error compiling Cython file:
------------------------------------------------------------
...
        
        # loop through dimensions
        for idim in prange(d_dimensions, nogil=True):

            # loop through the number of test points
            for iTest in prange(n_test, nogil=True):
                              ^
------------------------------------------------------------

/Users/eman/.ipython/cython/_cython_magic_029e0f0e560b830aefa06c993b2d5ad8.pyx:72:31: Trying to release the GIL while it was previously released.

Error compiling Cython file:
------------------------------------------------------------
...

            # loop through the number of test points
            for iTest in prange(n_test, nogil=True):

                # loop through the number of test points
                for iTrain in prange(n_train, nogil=True):
                                   ^
------------------------------------------------------------

/Users/eman/.ipython/cython/_cython_magic_0

### Making Numba Faster

In [36]:
@jit
def rbf_derivative_numba(x_train, x_function, weights, kernel_mat,
                   n_derivative=1, gamma=1.0):
    """This function calculates the rbf derivative
    Parameters
    ----------
    x_train : array, [N x D]
        The training data used to find the kernel model.

    x_function  : array, [M x D]
        The test points (or vector) to use.

    weights   : array, [N x D]
        The weights found from the kernel model
            y = K * weights

    kernel_mat: array, [N x M], default: None
        The rbf kernel matrix with the similarities between the test
        points and the training points.

    n_derivative : int, (default = 1) {1, 2}
        chooses which nth derivative to calculate

    gamma : float, default: None
        the parameter for the rbf_kernel matrix function

    Returns
    -------

    derivative : array, [M x D]
        returns the derivative with respect to training points used in
        the kernel model and the test points.

    Information
    -----------
    Author: Juan Emmanuel Johnson
    Email : jej2744@rit.edu
            juan.johnson@uv.es
    """

    # initialize the derivative
    d_dimensions = x_function.shape[1]
    n_test = x_function.shape[0]
    n_train = x_train.shape[0]
    
    derivative = np.zeros((n_test, d_dimensions))

    # consolidate the parameters
    theta = 2.0 * gamma


    if n_derivative == 1:
        
        # loop through dimensions
        for idim in np.arange(0, d_dimensions):

            # loop through the number of test points
            for iTest in np.arange(0, n_test):

                # loop through the number of test points
                for iTrain in np.arange(0, n_train):

                    # calculate the derivative for the test points
                    derivative[iTest, idim] += theta * weights[iTrain] * \
                                              (x_train[iTrain, idim] -
                                               x_function[iTest, idim]) * \
                                              kernel_mat[iTrain, iTest]
                        
    # 2nd derivative
    elif n_derivative == 2:

        # loop through dimensions
        for dim in np.arange(0, d_dimensions):

            # loop through the number of test points
            for iTest in np.arange(0, n_test):

                # loop through the number of test points
                for iTrain in np.arange(0, n_train):
                    derivative[iTest, dim] += weights[iTrain] * \
                                              (theta ** 2 *
                                               (x_train[iTrain, dim] - x_function[iTest, dim]) ** 2
                                               - theta) * \
                                              kernel_mat[iTrain, iTest]

    return derivative

In [37]:
temp_d = rbf_derivative_numba(np.int64(x_train), 
                              np.int64(x_test), 
                              kernel_mat=np.float64(kernel), 
                              weights=np.float64(weights),
                              n_derivative=n_derivative,
                              gamma=gamma)

In [38]:
%%timeit
rbf_derivative_numba(np.int64(x_train), 
                              np.int64(x_test), 
                              kernel_mat=np.float64(kernel), 
                              weights=np.float64(weights),
                              n_derivative=n_derivative,
                              gamma=gamma)

1.39 s ± 50.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [140]:
rbf_derivative_numba.inspect_types()

rbf_derivative_numba (array(int64, 2d, C), array(int64, 2d, C), array(float64, 2d, C), array(float64, 2d, C), int64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-138-76c90ea42936>
# --- LINE 1 --- 
# label 0
#   del $const0.3
#   del $0.2
#   del $0.4
#   del $const0.7
#   del $0.6
#   del $0.8
#   del $const0.11
#   del $0.10
#   del $0.12
#   del $0.13
#   del $0.17
#   del $0.14
#   del $0.18
#   del gamma
#   del $const0.19
#   del $0.21
#   del $const0.23

@jit

# --- LINE 2 --- 

def rbf_derivative_numba(x_train, x_function, weights, kernel_mat,

                   # --- LINE 3 --- 

                   n_derivative=1, gamma=1.0):

    # --- LINE 4 --- 

    """This function calculates the rbf derivative

    # --- LINE 5 --- 

    Parameters

    # --- LINE 6 --- 

    ----------

    # --- LINE 7 --- 

    x_train : array, [N x D]

        # --- LINE 8 --- 

        The training data used to find the kernel mode

In [141]:
print('Numba w/ Jit:')
jitted = %timeit -o rbf_derivative_numba(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)


Numba w/ Jit:
1.44 s ± 26.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
fast_numba = jit()(rbf_derivative)

print('Numba w/ Jit:')
%timeit fast_numba(x_train, x_test, kernel_mat=kernel, weights=weights, gamma=gamma)

Numba w/ Jit:
1.62 s ± 58.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [53]:
rbf_derivative_numba = jit(void(double[:], double[:,:], double[:,:], double[:,:], double[:]))(rbf_derivative_numba)

AttributeError: 'CPUDispatcher' object has no attribute '__defaults__'

In [28]:
%timeit rbf_derivative_numba(x_train, x_test, kernel_mat=kernel, weights=weights, n_derivative=1, gamma=gamma)

UntypedAttributeError: Failed at nopython (nopython frontend)
Unknown attribute 'shape' of type Module(<module 'numpy' from '/Users/eman/anaconda3/lib/python3.6/site-packages/numpy/__init__.py'>)
File "<ipython-input-26-6afc3b98ed16>", line 42
[1] During: typing of get attribute at <ipython-input-26-6afc3b98ed16> (42)

13.9 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
print(x_train.shape, x_train.dtype)
print(x_test.shape, x_test.dtype)
print(kernel.shape, kernel.dtype)
print(weights.squeeze().shape, weights.squeeze().dtype)
print(gamma.shape, gamma.dtype, gamma.astype(np.float).dtype)

(700, 1) int64
(300, 1) int64
(700, 700) float64
(700,) float64
() float64 float64
